In [2]:
import pandas as pd 

In [4]:
df=pd.read_csv('../dataset/train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
##Drpping the null value 
df=df.dropna()

In [7]:
## Get the independent feature
X=df.drop('label', axis=1)

In [8]:
## Get the dependent feature
y=df['label']

In [9]:
## checking inbalance dataset 
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf 
tf.__version__

'2.19.0'

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [16]:
##vocabulary Size 
voc_size=5000

In [17]:
messages=X.copy()

In [18]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
messages.reset_index(inplace=True)

In [25]:
import nltk 
import re 
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print('LENGTH',len(stop_words), 'STOPWORDS',stop_words[:] )  # Example: print first 10 stopwords
print(len(stop_words[:]))  # Example: print first 10 stopwords

LENGTH 198 STOPWORDS ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', '

In [26]:
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sopan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
## Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer() 
corpus = []
for i in range(0,len(messages)): 
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review=review.lower() 
    review=review.split() 
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]

    # review=[ps.stem(word) for word in review if not  word in stopwords.words('english')] 
    review= ' '.join(review) 
    corpus.append(review)

In [35]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [37]:
onehot_repr=[one_hot(words, voc_size) for words in corpus] 
onehot_repr

[[573, 3649, 3515, 1444, 4922, 1950, 3721, 2571, 1559, 1661],
 [2604, 1180, 4733, 133, 3563, 4038, 4366],
 [2488, 215, 1330, 48],
 [3993, 2530, 76, 1719, 3213, 4726],
 [4372, 3563, 677, 601, 2286, 4884, 3563, 972, 215, 2630],
 [2651,
  3239,
  4638,
  3432,
  4543,
  1805,
  4764,
  275,
  3109,
  3378,
  4630,
  3202,
  4657,
  4426,
  4366],
 [3923, 3385, 3149, 4766, 4214, 267, 1977, 303, 1037, 1294, 2936],
 [1406, 4325, 2859, 3472, 4503, 1871, 1805, 1279, 1037, 1294, 2936],
 [2857, 2867, 4272, 2956, 4415, 4326, 4568, 2001, 1805, 371],
 [1217, 2512, 2394, 2477, 2720, 3612, 1657, 1207],
 [1580, 4248, 621, 3847, 4771, 2625, 1148, 1156, 4951, 1324, 1177],
 [1719, 3081, 4922, 4326, 1805, 4503],
 [152, 3841, 1878, 273, 1211, 2121, 4589, 2318, 3491],
 [4106, 2197, 4134, 141, 1502, 809, 885, 1037, 1294, 2936],
 [1478, 499, 3669, 3186, 2578, 1037, 1294, 2936],
 [3565, 4240, 2974, 1177, 2094, 3637, 2113, 1469, 2198, 1801],
 [3242, 2473, 1180],
 [3984, 2843, 3247, 2519, 1805, 1723, 3911, 4366]

In [39]:
onehot_repr[0]

[573, 3649, 3515, 1444, 4922, 1950, 3721, 2571, 1559, 1661]

In [40]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

### Embadded Representation

In [43]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2571 1559 1661]
 [   0    0    0 ... 3563 4038 4366]
 [   0    0    0 ...  215 1330   48]
 ...
 [   0    0    0 ... 1037 1294 2936]
 [   0    0    0 ... 1246 1048 3739]
 [   0    0    0 ... 1191 4210  850]]


In [44]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  573,
       3649, 3515, 1444, 4922, 1950, 3721, 2571, 1559, 1661], dtype=int32)

In [57]:
#create model 
embedding_vector_feature=40
model=Sequential() 
model.add(Embedding(voc_size,embedding_vector_feature))
model.build(input_shape=(None,sent_length))
model.add(Bidirectional(LSTM(100))) 
model.add(Dense(1,activation='sigmoid')) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 200)                 │         112,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             201 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 313,001 (1.19 MB)

 Trainable params: 313,001 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

None


In [99]:
# Creating model
embedding_vector_features=40
model1=Sequential()
model.add(Embedding(voc_size,embedding_vector_feature))
model.build(input_shape=(None,sent_length))
model1.add(LSTM(100))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary()) 

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [59]:
len(embedded_docs),y.shape

(18285, (18285,))

In [60]:
import numpy as np 
X_final=np.array(embedded_docs) 
y_final=np.array(y)

In [62]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [69]:
# from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [70]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 22s 61ms/step - accuracy: 0.7950 - loss: 0.4221 - val_accuracy: 0.9153 - val_loss: 0.1965
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 57ms/step - accuracy: 0.9455 - loss: 0.1393 - val_accuracy: 0.9181 - val_loss: 0.2265
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 60ms/step - accuracy: 0.9621 - loss: 0.0948 - val_accuracy: 0.9157 - val_loss: 0.2086
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 51ms/step - accuracy: 0.9801 - loss: 0.0635 - val_accuracy: 0.9095 - val_loss: 0.2787
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 53ms/step - accuracy: 0.9866 - loss: 0.0430 - val_accuracy: 0.9069 - val_loss: 0.3084
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.9914 - loss: 0.0296 - val_accuracy: 0.9062 - val_loss: 0.3500
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step - accuracy: 0.9933 - loss: 0.0217 - val_accuracy: 0.9064 - val_loss: 0.3776
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 51ms/step - accuracy: 0.9967 - loss: 0.0129 - 

### Performance Metrics And Accuracy

In [90]:
y_pred = model.predict_on_batch(X_test)

In [91]:
from sklearn.metrics import confusion_matrix

In [94]:
y_pred_labels = (y_pred > 0.5).astype(int)  # threshold at 0.5
confusion_matrix(y_test,y_pred_labels)

array([[3097,  322],
       [ 251, 2365]])

In [95]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_labels)

0.9050538525269263

In [96]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_labels))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.90      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.91      0.91      0.91      6035

